Reduz um dataset e gera um arquivo de tag de acordo (tags devem estar no formato base)

In [115]:
import sys
import os
import glob
from shutil import copyfile
import shutil
import numpy as np
import json

TAGS_PATH = os.path.join('.','tags')
DATA_PATH = os.path.join('.','datasets')
DEST_PATH = os.path.join('.','reduced')
TOTAL_PHOTOS = 4617
TARGET = 2000
rate = TARGET/TOTAL_PHOTOS

In [111]:
def find_annotations(datsetnumber, files):
    annotations = []
    datsetnumber = str(datsetnumber)
    tag_path = os.path.join(TAGS_PATH,'{}.txt'.format(datsetnumber))
    if(not os.path.isfile(tag_path)):
        return annotations
    tags_file = open(tag_path, 'r', encoding='utf8')
    tags = tags_file.read()
    tags = tags.replace('.png', '.jpg')
    tags = tags.replace('.PNG', '.jpg')
    tags = tags.split('\n')[1:-1]
    for tag in tags:
        splitted = tag.split('|')
        filename = splitted[0]
        atype = splitted[1]
        splitted[0] = datsetnumber + '_' + splitted[0]
        if((filename in files) and (atype in ['ball', 'goal', 'robot'])):
            annotations.append('|'.join(splitted))
        elif((filename in files) and (atype in ['goalpost'])):
            if(splitted[2]=='not in image'):
                annotations.append('|'.join(splitted))
            else:
                vector = splitted[2][:-2]+'}'
                vector_json = json.loads(vector)
                vector_json.pop('x2', None)
                vector_json.pop('y2', None)
                vector_json.pop('x4', None)
                vector_json.pop('y4', None)
                vector_json['x2'] = vector_json.pop('x3', None)
                vector_json['y2'] = vector_json.pop('y3', None)
                splitted[2] = json.dumps(vector_json)
                annotations.append('|'.join(splitted))
    tags_file.close()
    return annotations

In [112]:
annotation=[]
count = 0
DatasetCounter = 0
def createNewPath():
    global DatasetCounter
    DatasetCounter += 1
    dataset_path = os.path.join(DEST_PATH, str(DatasetCounter))
    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path)
    return dataset_path
path = createNewPath()

In [116]:
folder_annotation = []
for root, dirs, files in os.walk(DATA_PATH):
    if len(files) > 0:
        len_target = len(files)*rate
        indices = np.linspace(0, len(files)-1, len_target,dtype=int)
        selected_files = [files[i] for i in indices]
        annotations = find_annotations(root.split('/')[-1], selected_files)
        folder_annotation.extend(annotations)
        for file in selected_files:
            img_path = os.path.join(root, file)
            img_dest = os.path.join(path, root.split('/')[-1]+'_'+file)
            copyfile(img_path, img_dest)
            count = count + 1
        if(count >= 950):
            annotation_path = os.path.join(DEST_PATH, str(DatasetCounter)+'.txt')
            annotation_file = open(annotation_path, 'w')
            annotation_file.write('\n'.join(folder_annotation))
            annotation_file.close()
            path = createNewPath()
            count = 0
            folder_annotation = []

/home/eduardo/anaconda3/envs/humanoid/lib/python3.5/site-packages/ipykernel_launcher.py:5: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  """


In [114]:
"""annotation_path = os.path.join(DEST_PATH, str(DatasetCounter)+'.txt')
annotation_file = open(annotation_path, 'w')
annotation_file.write('\n'.join(folder_annotation))
annotation_file.close()
path = createNewPath()
count = 0
folder_annotation = []"""